In [9]:



import feedparser as fp
import newspaper as np
import json
import os
import datetime
import subprocess

import requests
import json
import subprocess

# Télécharger les fichiers JSON depuis GitHub
def telecharger_fichier_github(url):
    reponse = requests.get(url)
    return reponse.json()

# Fonction pour récupérer les articles et mettre à jour la base de données
def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database):
    new_articles = []
    for source_url in flux_rss_a_lire:
        data = fp.parse(source_url)
        for item in data.entries:
            article = {
                "title": item.title,
                "date": item.published,
                "url": item.link,
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": flux_rss_a_lire[source_url]
            }
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                article["content"] += "\n\n" + article_obj.text
                article["author"] = article_obj.authors
                database[article["url"]] = article
    return database

# Sauvegarder la base de données mise à jour localement
def sauvegarder_fichier_local(fichier, contenu):
    with open(fichier, 'w') as f:
        json.dump(contenu, f, ensure_ascii=False, indent=4)

# Commit et push vers GitHub
def pousser_github():
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", "Mise à jour de la base de données"])
    subprocess.run(["git", "push"])

# Exécution du processus
sources = telecharger_fichier_github('https://github.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/blob/main/Data/tvinfo-sources.json')
database = telecharger_fichier_github('https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo.json?token=GHSAT0AAAAAACOKDDYSH575FCEVYQ4OJMLKZOSLHXQ')

database_mise_a_jour = mettre_a_jour_base_de_donnees(sources, database)
sauvegarder_fichier_local('Data/fichier.json', database_mise_a_jour)

# Assurez-vous d'être dans le bon dossier contenant votre dépôt Git localement
pousser_github()


JSONDecodeError: Expecting value: line 7 column 1 (char 6)